In [74]:
import pandas as pd
import icecream as ic
import numpy as np
import plotly
import matplotlib as plt
import xml.etree.ElementTree as ET
from inspect import getmembers, isclass, isfunction
import pandas as pd
import itertools
from rdkit import Chem

In [84]:
#use base environment
import pandas as pd
import icecream as ic
import numpy as np
import plotly
import matplotlib as plt
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
# from chemicalchecker import ChemicalChecker
# cc = ChemicalChecker()

from standardiser import unsalt, neutralise,rules
from standardiser.rules_demo import show_change

import e3fp
from e3fp.pipeline import confs_from_smiles

import six

import warnings

import sys
import re
import random
 
from ipywidgets import HTML

import pandas as pd
 
from rdkit import Chem
from rdkit.Chem import Draw, PandasTools, AllChem
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.molSize = (450, 200)
PandasTools.RenderImagesInAllDataFrames()

#import setting from Standardizer
from __future__  import print_function, division, absolute_import

import six

import warnings

import sys
import re
import random
 
# from ipywidgets import HTML

import pandas as pd
 
from rdkit import Chem
from rdkit.Chem import Draw, PandasTools, AllChem
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.molSize = (450, 200)
PandasTools.RenderImagesInAllDataFrames()

from urllib.request import urlopen
from urllib.parse import quote



All Categories potentially needed from the paper: <br>
<ul>
<li>mechanism-of-action
<li>metabolism
<li>pharmacodynamics
<li>absorption
<li>protein-binding
<li>atc-codes
<li>drug-interactions
<li>Target

<li>Enzyme
<li>Carrier
<li>transporter
<li>drug-interactions
<li>external-identifiers - chemeble

In [4]:
chemical_list = ['Acetazolamide','Acetylcholine','Adenosine']

In [5]:
drugbank_df = pd.read_csv("DB/drugbank_extracted.csv")
drugbank_query = drugbank_df[drugbank_df["dg_name"].isin(chemical_list)]
chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)
chemble_B2 = pd.read_csv("../chembl31forlinfeng/B2_data.csv", sep=',', header=0)
ecod_df = pd.read_csv('../B3_ecod.latest.domains.csv', header=4)
B5 = pd.read_csv('../B5.tsv',sep='\t')
bindingdb_df = pd.read_csv('DB/Bindingdb_IC50_human_cleaned_new.csv')

/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_24065/2179903639.py:3: DtypeWarning: Columns (5,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)
/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_24065/2179903639.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  chemble_B2 = pd.read_csv("../chembl31forlinfeng/B2_data.csv", sep=',', header=0)


In [80]:
def CIRconvert(ids): #convert name to smile
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'

B1: mechanism of action

In [26]:
# mechanism of action. Mechanisms of action of approved and experimental drugs were collected from DrugBank53 (v.4, https://www.drugbank.ca) by selecting those protein targets with a known pharmacological action and action mode. 
#mechanism of action
# indices = []
# for x in ['mechanism of action','target_name','action','atc_codes']:
#     uniqueValues, indicesList = np.unique(drugbank_query[x], return_index=True)
#     indices.append(indicesList)
# indices

def b1_func(drugbank_query, chemble_B1_compound):
    db = []
    mechanism_of_action = [str(x) for x in drugbank_query["mechanism_of_action"].unique().tolist()]
    mechanism_of_action= ' '.join(mechanism_of_action)
    db.append(mechanism_of_action)
    target_name = [str(x) for x in drugbank_query["target_name"].unique().tolist()]
    target_name = ' '.join(target_name)
    db.append(target_name)
    action = [str(x) for x in drugbank_query["action"].unique().tolist()]
    action = ' '.join(action)
    db.append(action)
    
    # chemble_B1_compound = chemble_B1[chemble_B1['compound_name']==chemical].iloc[:,4:14]

    chemble_B1_compound_list = chemble_B1_compound.values.tolist()
    chemble_B1_compound_list = list(itertools.chain.from_iterable(chemble_B1_compound_list))
    cleanedList = [x for x in chemble_B1_compound_list if str(x) != 'nan']
    cleanedList = " ".join(cleanedList)
    db.append(cleanedList)
    db = " ".join(db)

    return db

    # #meta
    # print('metabolism')
    # print(drugbank_query["metabolism"].unique())

    # print("="*20)

    # #abs
    # print('absorption')
    # print(drugbank_query["absorption"].unique())

    # print("="*20)

B2: metabolic genes

In [22]:
#  drug-metabolizing enzymes, transporters and carriers from DrugBank.
def b2_func(x, chemble_B1_compound, drugbank_query):
    # drugbank_df_query = drugbank_query[drugbank_query["dg_name"]==chemical]
    db = []
    
    transporters = [str(x) for x in drugbank_query["transporters"].unique().tolist()]
    transporters = ' '.join(transporters)

    enzymes = [str(x) for x in drugbank_query["enzymes"].unique().tolist()]
    enzymes = ' '.join(enzymes)

    carriers = [str(x) for x in drugbank_query["carriers"].unique().tolist()]
    carriers = ' '.join(carriers)
    
    # chemble_B1_compound = chemble_B1[chemble_B1['compound_name']==chemical].iloc[:,4:14]
    metabolic_protein = chemble_B2[chemble_B2['compound_name']==x]['uniprot_id'].unique()
    chemble_B1_compound = chemble_B1_compound[chemble_B1_compound['uniprot_id'].isin(metabolic_protein)]
    chemble_B1_compound_list = chemble_B1_compound.values.tolist()
    chemble_B1_compound_list = list(itertools.chain.from_iterable(chemble_B1_compound_list))
    cleanedList = [x for x in chemble_B1_compound_list if str(x) != 'nan']
    cleanedList = " ".join(cleanedList)
    db.append(cleanedList)

    return db

In [18]:
metabolic_protein = chemble_B2[chemble_B2['compound_name']=='Acetazolamide']['uniprot_id'].unique()
chemble_B1_compound[chemble_B1_compound['uniprot_id'].isin(metabolic_protein)]

,uniprot_id,action_type,l1,l2,l3,l4,l5,l6,l7,l8
22307,P00915,INHIBITOR,Enzyme,Lyase,NaN,NaN,NaN,NaN,NaN,NaN
22334,P00918,INHIBITOR,Enzyme,Lyase,NaN,NaN,NaN,NaN,NaN,NaN
22361,Q16790,INHIBITOR,Enzyme,Lyase,NaN,NaN,NaN,NaN,NaN,NaN
22388,Q9D6N1,INHIBITOR,Enzyme,Lyase,NaN,NaN,NaN,NaN,NaN,NaN
24705,Q95323,INHIBITOR,Enzyme,Lyase,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
11389785,O15438,INHIBITOR,Transporter,Primary active transporter,ATP-binding cassette,ABCC subfamily,NaN,NaN,NaN,NaN
11399766,O15439,INHIBITOR,Transporter,Primary active transporter,ATP-binding cassette,ABCC subfamily,NaN,NaN,NaN,NaN
11417735,Q8NFI3,INHIBITOR,Enzyme,Hydrolase,NaN,NaN,NaN,NaN,NaN,NaN
11759267,Q05769,INHIBITOR,Enzyme,Oxidoreductase,NaN,NaN,NaN,NaN,NaN,NaN


B3: crystals

In [ ]:
ligand_df

,inchi,ligand_index,ligand_name
0,"InChI=1S/C2H4O3/c1-5-2(3)4/h1H3,(H,3,4)",000,methyl hydrogen carbonate
1,InChI=1S/C35H42F2N2O6/c1-42-30-22-27(23-31(43-...,001,"1-[2,2-difluoro-2-(3,4,5-trimethoxy-phenyl)-ac..."
2,InChI=1S/C23H35N3O6/c1-5-15(4)20(22(29)24-18(2...,002,n-[(2r)-2-benzyl-4-(hydroxyamino)-4-oxobutanoy...
3,InChI=1S/C26H25N5O2/c1-17(2)15-30-24-22(25(32)...,003,5-methyl-7-(2-methylpropyl)-2-(naphthalen-1-yl...
4,InChI=1S/C8H9NO2/c9-7(8(10)11)6-4-2-1-3-5-6/h1...,004,(2s)-amino(phenyl)ethanoic acid
...,...,...,...
39108,"InChI=1S/C20H18FN3O4S/c1-23(29(2,27)28)18-14-4...",ZZV,"n-[7-(4-fluorobenzyl)-9-hydroxy-8-oxo-7,8-dihy..."
39109,InChI=1S/C19H14FN3O2/c20-13-7-5-12(6-8-13)11-2...,ZZW,9-(4-fluorobenzyl)-n-hydroxy-9h-beta-carboline...
39110,InChI=1S/C21H21ClFN5O4/c1-4-26-8-10(2)28-16-14...,ZZX,(6s)-2-(3-chloro-4-fluorobenzyl)-8-ethyl-10-hy...
39111,InChI=1S/C14H10N4O5S/c15-14(19)9-7-12-10(16-8-...,ZZY,"1-[(2-nitrophenyl)sulfonyl]-1h-pyrrolo[3,2-b]p..."


In [9]:
drugbank_df_query = drugbank_query[drugbank_query["dg_name"]=='Acetazolamide']
db = []
pdb_list = drugbank_query['pdb_list'].unique().tolist()[0].split(';')

ligand_df = pd.read_csv('../B3_ligand-inchi.ich.txt', sep='\t', names=['inchi', 'ligand_index', 'ligand_name'])
ligand_index_df = pd.read_csv('../B3_ligand-pdb_index.tdd.txt', sep='\t', names=['ligand_index', 'pdf_no'])
# http://ligand-expo.rcsb.org/ld-download.html
#get the ligand_id from pdb
ligand_id = []

for index, row in ligand_index_df.iterrows():
    repo_pdf = row['pdf_no'].split(' ')
    if bool(set(repo_pdf) & set(pdb_list)):
        ligand_id.append(row['ligand_index'])
        
ligands = ligand_df[ligand_df['ligand_index'].isin(ligand_id)]['inchi'] #get the inchi of the ligand

In [28]:
def b3_func(drugbank_df_query):
    db = []
    pdb_list = drugbank_query['pdb_list'].unique().tolist()[0].split(';')

    ligand_df = pd.read_csv('../B3_ligand-inchi.ich.txt', sep='\t', names=['inchi', 'ligand_index', 'ligand_name'])
    ligand_index_df = pd.read_csv('../B3_ligand-pdb_index.tdd.txt', sep='\t', names=['ligand_index', 'pdf_no'])
    # http://ligand-expo.rcsb.org/ld-download.html
    #get the ligand_id from pdb
    ligand_id = []

    for index, row in ligand_index_df.iterrows():
        repo_pdf = row['pdf_no'].split(' ')
        if bool(set(repo_pdf) & set(pdb_list)):
            ligand_id.append(row['ligand_index'])
            
    ligand_df[ligand_df['ligand_index'].isin(ligand_id)]['inchi'] #get the inchi of the ligand
    str_list = [str(x) for x in ligands.tolist()]
    str_list = ' '.join(str_list)
    db.append(str_list)
        
    ecod_pdb = ecod_df[ecod_df['pdb'].isin(pdb_list)][['x_name', 'h_name', 't_name', 'f_name']].values.tolist()
    ecod_pdb = list(np.concatenate(ecod_pdb).flat)

    ecod_pdb = [str(x) for x in ecod_pdb]
    ecod_pdb = ' '.join(ecod_pdb)
    db.append(ecod_pdb)
    db = ' '.join(db)
    return db

B4: binding


In [33]:
chemble_clean_IC50 = pd.read_csv("../chembl31_ic50_clean/Chembl31_IC50_human_cleaned_new.csv", sep=',', header=0)

In [13]:
#! ‘binding’ type, related to ‘single proteins
chemble_B4B5 = pd.read_csv("../chembl31forlinfeng/B4B5_data.csv", sep=',', header=0)

/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_24065/991561538.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  chemble_B4B5 = pd.read_csv("../chembl31forlinfeng/B4B5_data.csv", sep=',', header=0)


In [16]:
ion_channels = ['Ligand-gated ion channel','Voltage-gated ion channel', 'Other ion channel']
g_protein = ['Family A G protein-coupled receptor', 'Family C G protein-coupled receptor',
            'Family B G protein-coupled receptor','Frizzled family G protein-coupled receptor',
            'Taste family G protein-coupled receptor']

not_other = ['Ligand-gated ion channel','Voltage-gated ion channel', 'Other ion channel'
            'Family A G protein-coupled receptor', 'Family C G protein-coupled receptor',
            'Family B G protein-coupled receptor','Frizzled family G protein-coupled receptor',
            'Taste family G protein-coupled receptor', 'Kinase','Nuclear receptor']
# kinases ≤30 nM, 
kinase_df = chemble_B1_compound[chemble_B1_compound['l2']=="Kinase"]

# G protein-coupled receptors ≤100 nM, 
g_protein_df = chemble_B1_compound[chemble_B1_compound['l2'].isin(g_protein)]

# nuclear receptors ≤100 nM,  
nuclear_receptor_df = chemble_B1_compound[chemble_B1_compound['l2']=='Nuclear receptor']

# ion channels ≤10 μM
ion_channel_df = chemble_B1_compound[chemble_B1_compound['l2'].isin(ion_channels)]

# others ≤1 μM. 
others_df = chemble_B1_compound[~chemble_B1_compound['l2'].isin(not_other)]

# We also kept activities one order of magnitude lower than the class-specific cutoff (to a maximum of 10 μM), 
# and gave these annotations half the weight in downstream analyses (that is, log10 scaling)

NameError: name 'chemble_B1_compound' is not defined

In [ ]:
# chemble_B4B5 = chemble_B4B5[chemble_B4B5['standard_units'].isin(['nM','uM'])]
chemble_B4B5_compound = chemble_B4B5[chemble_B4B5['compound_name']=='Acetazolamide']
chemble_B4B5_compound_target = chemble_B4B5_compound[chemble_B4B5_compound['uniprot_id'].isin(chemble_B1_compound['uniprot_id'].unique())] #B5
chemble_B4B5_compound_target_ic50 = chemble_B4B5_compound_target [chemble_B4B5_compound_target ['standard_type']=='IC50'] #B4

In [ ]:
kinase_df.head()

,uniprot_id,action_type,l1,l2,l3,l4,l5,l6,l7,l8
6154169,P17252,INHIBITOR,Enzyme,Kinase,Protein Kinase,AGC protein kinase group,AGC protein kinase PKC family,AGC protein kinase PKC alpha subfamily,NaN,NaN
6154196,P27361,INHIBITOR,Enzyme,Kinase,Protein Kinase,CMGC protein kinase group,CMGC protein kinase MAPK family,CMGC protein kinase ERK subfamily,NaN,NaN
6154223,P28482,INHIBITOR,Enzyme,Kinase,Protein Kinase,CMGC protein kinase group,CMGC protein kinase MAPK family,CMGC protein kinase ERK subfamily,NaN,NaN
6154250,Q16539,INHIBITOR,Enzyme,Kinase,Protein Kinase,CMGC protein kinase group,CMGC protein kinase MAPK family,CMGC protein kinase p38 subfamily,NaN,NaN
6154304,P00533,INHIBITOR,Enzyme,Kinase,Protein Kinase,TK protein kinase group,Tyrosine protein kinase EGFR family,NaN,NaN,NaN


In [62]:
bindingdb_df_compound[bindingdb_df_compound['UNIPROT_ID'].isin(query)]

NameError: name 'bindingdb_df_compound' is not defined

In [108]:
def b4_func(chemble_B4B5_compound_target_ic50, bindingdb_df_compound):
    proteins = []

    from rdkit.Chem.inchi import MolToInchiKey

        
    kinase_ic = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['uniprot_id'].isin(kinase_df['uniprot_id'])]
    kinase_ic_filter = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['standard_value']<=30]
    # kinase_ic_halfweight = chemble_B4B5_compound_target_ic50[(chemble_B4B5_compound_target_ic50['standard_value']>30) & (chemble_B4B5_compound_target_ic50['standard_value']<= 10000)]
    kinase = ' '.join(kinase_ic_filter['uniprot_id'].values.tolist())
    proteins.append(kinase)
    
    query = kinase_df['uniprot_id']
    binding_kinase = bindingdb_df_compound[bindingdb_df_compound['INCHI_KEY']=='BZKPWHYZMXOIDC-UHFFFAOYSA-N']['values'].values.tolist()
    binding_kinase = [str(x) for x in binding_kinase]
    binding_kinase = " ".join(binding_kinase)
    proteins.append(binding_kinase)

    g_protein_ic = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['uniprot_id'].isin(g_protein_df['uniprot_id'])]
    g_protein_ic_filter = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['standard_value']<=100]
    # g_protein_ic_halfweight = chemble_B4B5_compound_target_ic50[(chemble_B4B5_compound_target_ic50['standard_value']>100) & (chemble_B4B5_compound_target_ic50['standard_value']<= 10000)]
    g_protein = ' '.join(g_protein_ic_filter['uniprot_id'].values.tolist())
    proteins.append(g_protein)
    
    query = g_protein_df['uniprot_id']
    binding_kinase = bindingdb_df_compound[bindingdb_df_compound['INCHI_KEY']=='BZKPWHYZMXOIDC-UHFFFAOYSA-N']['values'].values.tolist()
    binding_kinase = [str(x) for x in binding_kinase]
    binding_kinase = " ".join(binding_kinase)
    proteins.append(binding_kinase)

    nuclear_receptor_ic = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['uniprot_id'].isin(nuclear_receptor_df['uniprot_id'])].values.tolist()
    nuclear_receptor_ic_filter = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['standard_value']<=100]
    # nuclear_receptor_ic_halfweight = chemble_B4B5_compound_target_ic50[(chemble_B4B5_compound_target_ic50['standard_value']>100) & (chemble_B4B5_compound_target_ic50['standard_value']<= 10000)]
    nuclear_receptor = ' '.join(nuclear_receptor_ic_filter['uniprot_id'].values.tolist())
    proteins.append(nuclear_receptor)
    
    query = nuclear_receptor_df['uniprot_id']
    binding_kinase = bindingdb_df_compound[bindingdb_df_compound['INCHI_KEY']=='BZKPWHYZMXOIDC-UHFFFAOYSA-N']['values'].values.tolist()
    binding_kinase = [str(x) for x in binding_kinase]
    binding_kinase = " ".join(binding_kinase)
    proteins.append(binding_kinase)

    

    ion_channel_ic = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['uniprot_id'].isin(ion_channel_df['uniprot_id'])]
    ion_channel_ic_filter = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['standard_value']<=1000]
    # ion_channel_ic_halfweight = chemble_B4B5_compound_target_ic50[(chemble_B4B5_compound_target_ic50['standard_value']>1000) & (chemble_B4B5_compound_target_ic50['standard_value']<= 10000)]
    ion_channel_protein = ' '.join(ion_channel_ic_filter['uniprot_id'].values.tolist())
    proteins.append(ion_channel_protein)
    
    query = ion_channel_df['uniprot_id']
    binding_kinase = bindingdb_df_compound[bindingdb_df_compound['INCHI_KEY']=='BZKPWHYZMXOIDC-UHFFFAOYSA-N']['values'].values.tolist()
    binding_kinase = [str(x) for x in binding_kinase]
    binding_kinase = " ".join(binding_kinase)
    proteins.append(binding_kinase)


    other_ic = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['uniprot_id'].isin(others_df['uniprot_id'])] #!数据重复是怎么回事
    other_ic_filter = other_ic[other_ic['standard_value']<=1000]
    # other_ic_halfweight = other_ic[(other_ic['standard_value']>1000) & (chemble_B4B5_compound_target_ic50['standard_value']<= 10000)]
    other = ' '.join(other_ic_filter['uniprot_id'].values.tolist())
    proteins.append(other)
    
    query = others_df['uniprot_id']
    binding_kinase = bindingdb_df_compound[bindingdb_df_compound['UNIPROT_ID'].isin(query)] 
    binding_kinase = [str(x) for x in binding_kinase]
    binding_kinase = " ".join(binding_kinase)
    proteins.append(binding_kinase)

    proteins = " ".join(proteins)
    
    return proteins


In [60]:
#  Protein binding data were obtained from ChEMBL by searching for bioassays of ‘binding’ type, related to ‘single proteins’ with an experimental measure of standard type (‘pChEMBL’ value available).


#We also collected BindingDB records with activity expressed as concentrations (https://www.bindingdb.org, February 2017). 
#Data were discretized by applying the following activity cutoffs, recommended in Pharos17 (http://pharos.nih.gov/): kinases ≤30 nM, G protein-coupled receptors ≤100 nM, nuclear receptors ≤100 nM,  ion channels ≤10 μM and others ≤1 μM. 


# kept activities one order of magnitude lower than the class-specific cutoff (to a maximum of 10 μM), and gave these annotations half the weight in downstream analyses (that is, log10 scaling). 
# Finally, protein class hierarchy information was kept as in B1.
bindingdb_df = pd.read_csv('DB/Bindingdb_IC50_human_cleaned_new.csv')

In [ ]:
import ast
import re
txt = bindingdb_df_compound['values'][673858]
x = re.search("[0-9]+.[0-9]+(\s[0-9]+.[0-9]+)*", txt).group().split(' ')
np.mean(list(map(float,x)))

6.0215250000000005

B5: high-throughput screening bioassays

In [ ]:
#need data from chemble
# Bioactivity values from this repository were directly downloaded from ChEMBL,
# since the latter conveniently applies a processing pipeline that collects only confirmatory assays and maps related protein targets to UniProt identifiers. 
# Most of the assays belong to the ‘functional’ category.
# For completeness, we included other functional assays available in ChEMBL. 
# We chose a relaxed activity cutoff of 10 μM, or checked for the word ‘active’ in the description of the assay. We kept the protein class hierarchy as in B1

kinase_ic = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['uniprot_id'].isin(kinase_df['uniprot_id'])]
kinase_ic_filter = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['standard_value']]

In [113]:
def b5_func(chemble_B4B5_compound_target_ic50, bindingdb_df_compound):
    proteins = []

    from rdkit.Chem.inchi import MolToInchiKey
    query = CIRconvert('Acetazolamide')
    mol = Chem.MolFromSmiles(query)
    query = MolToInchiKey(mol)
        
    kinase_ic = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['uniprot_id'].isin(kinase_df['uniprot_id'])]
    kinase_ic_filter = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['standard_value']<=30]
    # kinase_ic_halfweight = chemble_B4B5_compound_target_ic50[(chemble_B4B5_compound_target_ic50['standard_value']>30) & (chemble_B4B5_compound_target_ic50['standard_value']<= 10000)]
    kinase = ' '.join(kinase_ic_filter['uniprot_id'].values.tolist())
    proteins.append(kinase)
    
    query = kinase_df['uniprot_id']
    binding_kinase = bindingdb_df_compound[bindingdb_df_compound['INCHI_KEY']=='BZKPWHYZMXOIDC-UHFFFAOYSA-N']['values'].values.tolist()
    binding_kinase = [str(x) for x in binding_kinase]
    binding_kinase = " ".join(binding_kinase)
    proteins.append(binding_kinase)

    g_protein_ic = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['uniprot_id'].isin(g_protein_df['uniprot_id'])]
    g_protein_ic_filter = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['standard_value']<=100]
    # g_protein_ic_halfweight = chemble_B4B5_compound_target_ic50[(chemble_B4B5_compound_target_ic50['standard_value']>100) & (chemble_B4B5_compound_target_ic50['standard_value']<= 10000)]
    g_protein = ' '.join(g_protein_ic_filter['uniprot_id'].values.tolist())
    proteins.append(g_protein)
    
    query = g_protein_df['uniprot_id']
    binding_kinase = bindingdb_df_compound[bindingdb_df_compound['INCHI_KEY']=='BZKPWHYZMXOIDC-UHFFFAOYSA-N']['values'].values.tolist()
    binding_kinase = [str(x) for x in binding_kinase]
    binding_kinase = " ".join(binding_kinase)
    proteins.append(binding_kinase)

    nuclear_receptor_ic = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['uniprot_id'].isin(nuclear_receptor_df['uniprot_id'])].values.tolist()
    nuclear_receptor_ic_filter = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['standard_value']<=100]
    # nuclear_receptor_ic_halfweight = chemble_B4B5_compound_target_ic50[(chemble_B4B5_compound_target_ic50['standard_value']>100) & (chemble_B4B5_compound_target_ic50['standard_value']<= 10000)]
    nuclear_receptor = ' '.join(nuclear_receptor_ic_filter['uniprot_id'].values.tolist())
    proteins.append(nuclear_receptor)
    
    query = nuclear_receptor_df['uniprot_id']
    binding_kinase = bindingdb_df_compound[bindingdb_df_compound['INCHI_KEY']=='BZKPWHYZMXOIDC-UHFFFAOYSA-N']['values'].values.tolist()
    binding_kinase = [str(x) for x in binding_kinase]
    binding_kinase = " ".join(binding_kinase)
    proteins.append(binding_kinase)

    

    ion_channel_ic = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['uniprot_id'].isin(ion_channel_df['uniprot_id'])]
    ion_channel_ic_filter = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['standard_value']<=1000]
    # ion_channel_ic_halfweight = chemble_B4B5_compound_target_ic50[(chemble_B4B5_compound_target_ic50['standard_value']>1000) & (chemble_B4B5_compound_target_ic50['standard_value']<= 10000)]
    ion_channel_protein = ' '.join(ion_channel_ic_filter['uniprot_id'].values.tolist())
    proteins.append(ion_channel_protein)
    
    query = ion_channel_df['uniprot_id']
    binding_kinase = bindingdb_df_compound[bindingdb_df_compound['INCHI_KEY']=='BZKPWHYZMXOIDC-UHFFFAOYSA-N']['values'].values.tolist()
    binding_kinase = [str(x) for x in binding_kinase]
    binding_kinase = " ".join(binding_kinase)
    proteins.append(binding_kinase)


    other_ic = chemble_B4B5_compound_target_ic50[chemble_B4B5_compound_target_ic50['uniprot_id'].isin(others_df['uniprot_id'])] #!数据重复是怎么回事
    other_ic_filter = other_ic[other_ic['standard_value']<=1000]
    # other_ic_halfweight = other_ic[(other_ic['standard_value']>1000) & (chemble_B4B5_compound_target_ic50['standard_value']<= 10000)]
    other = ' '.join(other_ic_filter['uniprot_id'].values.tolist())
    proteins.append(other)
    
    query = others_df['uniprot_id']
    binding_kinase = bindingdb_df_compound[bindingdb_df_compound['UNIPROT_ID'].isin(query)] 
    binding_kinase = [str(x) for x in binding_kinase]
    binding_kinase = " ".join(binding_kinase)
    proteins.append(binding_kinase)

    proteins = " ".join(proteins)
    
    return proteins


In [109]:
B5

,activity_id,standard_relation,standard_value,standard_units,standard_type,assay_id,description
0,31863,>,100000.0,nM,IC50,54505,Inhibitory concentration against human DNA top...
1,31864,=,2500.0,nM,IC50,83907,In vivo inhibitory activity against human Hepa...
2,31865,>,50000.0,nM,IC50,88152,In vivo concentration required against angioge...
3,31866,=,9000.0,nM,IC50,83907,In vivo inhibitory activity against human Hepa...
4,31867,NaN,NaN,nM,IC50,88153,In vivo concentration required against angioge...
...,...,...,...,...,...,...,...
19780364,24340580,=,74.0,%,Population Mitochondria Different-Phenotype (%),2035682,Multiplex assay against HEK293T
19780365,24340581,=,0.0,%,Population Membrane Permeable-Phenotype (%),2035682,Multiplex assay against HEK293T
19780366,24340582,=,256.0,%,Control DMSO Total Cells,2035682,Multiplex assay against HEK293T
19780367,24340583,=,33.0,%,Control DMSO Apoptotic Cells (%),2035682,Multiplex assay against HEK293T


In [ ]:
np.savetxt('test.csv',chemble_B4B5['standard_value'].unique(), delimiter=',')

Running to get output

In [114]:
B1_result = []
B2_result = []
B3_result = []
B4_result = []
B5_result = []

for x in chemical_list:    
    drugbank_df_query = drugbank_query[drugbank_query["dg_name"]==x]
    chemble_B1_compound = chemble_B1[chemble_B1['compound_name']==x].iloc[:,4:14]
    ion_channels = ['Ligand-gated ion channel','Voltage-gated ion channel', 'Other ion channel']
    g_protein = ['Family A G protein-coupled receptor', 'Family C G protein-coupled receptor',
                'Family B G protein-coupled receptor','Frizzled family G protein-coupled receptor',
                'Taste family G protein-coupled receptor']
    not_other = ['Ligand-gated ion channel','Voltage-gated ion channel', 'Other ion channel'
                'Family A G protein-coupled receptor', 'Family C G protein-coupled receptor',
                'Family B G protein-coupled receptor','Frizzled family G protein-coupled receptor',
                'Taste family G protein-coupled receptor', 'Kinase','Nuclear receptor']
    # kinases ≤30 nM, 
    kinase_df = chemble_B1_compound[chemble_B1_compound['l2']=="Kinase"]
    # G protein-coupled receptors ≤100 nM, 
    g_protein_df = chemble_B1_compound[chemble_B1_compound['l2'].isin(g_protein)]
    # nuclear receptors ≤100 nM,  
    nuclear_receptor_df = chemble_B1_compound[chemble_B1_compound['l2']=='Nuclear receptor']
    # ion channels ≤10 μM
    ion_channel_df = chemble_B1_compound[chemble_B1_compound['l2'].isin(ion_channels)]
    # others ≤1 μM. 
    others_df = chemble_B1_compound[~chemble_B1_compound['l2'].isin(not_other)]    
    
    chemble_B4B5_compound = chemble_B4B5[chemble_B4B5['compound_name']==x]
    chemble_B4B5_compound_target = chemble_B4B5_compound[chemble_B4B5_compound['uniprot_id'].isin(chemble_B1_compound['uniprot_id'].unique())] #B5
    chemble_B4B5_compound_target_ic50 = chemble_B4B5_compound_target [chemble_B4B5_compound_target ['standard_type']=='IC50'] #B4
        
    query = CIRconvert(x)
    mol = Chem.MolFromSmiles(query)
    query = MolToInchiKey(mol)
    bindingdb_df_compound = bindingdb_df[bindingdb_df['INCHI_KEY'] == query] #! what does the value here mean? It's different from the value from chemble, what test is used?

    B1 = b1_func(drugbank_query, chemble_B1_compound)
    B1_result.append(B1)
    # B1_result.append(morgan)
    B2 = b2_func(x, chemble_B1_compound, drugbank_query)
    B2_result.append(B2)
    
    B3 = b3_func(drugbank_df_query)
    B3_result.append(B3)

    # B4_result.append(keys)
    B4 = b4_func(chemble_B4B5_compound_target_ic50, bindingdb_df_compound)
    B4_result.append(B4)
    
    # B5_result.append(physicochemical)
    B5 = b5_func(chemble_B4B5_compound_target, bindingdb_df_compound)
    B5_result.append(B5)


Mechanisms of action of approved and experimental drugs were collected from 

DrugBank
protein targets with a known pharmacological action and action mode. 
drug-metabolizing enzymes, transporters and carriers

ChEMBL
‘activation’ modes (agonist, activator and so on)
‘inhibition’ modes (antagonist, competitor and so on)
identity of the protein targets
protein class memberships (G protein-coupled receptors, kinases and so on) at all levels of the ChEMBL target hierarchy. 
proteins involved in drug metabolism
protein class information
protein class hierarchy information
bioassays of ‘binding’ type

Protein Data Bank
Protein structures bound to each small molecule were then annotated with family (F and T groups) and 
superfamily (H and X groups) information, following the Evolutionary Classification of Protein Domains
‘single proteins’ with an experimental measure of standard type (‘pChEMBL’ value available)

BindingDB
activity expressed as concentrations
Data were discretized by applying the following activity cutoffs
kinases≤30 nM, G protein-coupled receptors ≤100 nM, nuclear receptors ≤100 nM, ion channels ≤10 μM and others ≤1 μM. 
We also kept activities one order of magnitude lower than the class-specific cutoff (to a maximum of 10 μM), and gave these annotations half the weight in downstream analyses (that is, log10 scaling). 

PubChem
Bioactivity values - activity cutoff of 10 μM
